In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv('/content/01JS2AFJE4CT7811VY4M714QVK.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/01JS2AFJE4CT7811VY4M714QVK.csv'

In [ ]:
df.info()

In [ ]:
df.Gender.value_counts()

In [ ]:
from os import replace
df.Gender.replace({'Female':0,'Male':1},inplace=True)
df.Gender.replace({'Femal':0,'M':1},inplace=True)
df.Gender.replace('Femail',0,inplace=True)


In [ ]:
df['Gender'].fillna(0,inplace=True)

In [ ]:
df.Gender.replace(0,'Female',inplace=True)
df.Gender.replace(1,'Male',inplace=True)


In [ ]:
df

In [ ]:
df['Total'].fillna(df['Total'].mean(),inplace=True)

In [ ]:
df.info()

In [ ]:
# prompt: convert object to string

df['Gender'] = df['Gender'].astype(str)
df['Payment Method'] = df['Payment Method'].astype(str)
df['Payment Method'] = df['Payment Method'].astype(str)

In [ ]:
df['Date']=pd.to_datetime(df['Date'],format='%d/%m/%Y',errors='coerce')

In [ ]:
df['Customer Name']=df['Customer Name'].astype('string')
df['Payment Method']=df['Payment Method'].astype('string')
df['Branch']=df['Branch'].astype('string')
df['Gender']=df['Gender'].astype('string')
df['Product']=df['Product'].astype('string')


In [ ]:
df['Invoice ID'].duplicated().sum()

In [ ]:
df.to_csv('processed_data.csv', index=False)


In [ ]:
# prompt: make Invoice ID unique

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import replace

df = pd.read_csv('/content/01JS2AFJE4CT7811VY4M714QVK.csv')

# ... (your existing code) ...

df['Invoice ID'].duplicated().sum()

# Generate unique Invoice IDs
df['Invoice ID'] = pd.factorize(df['Invoice ID'])[0] + 1  # Start from 1

# Check for duplicates again (should be 0)
print(df['Invoice ID'].duplicated().sum())

df.to_csv('processed_data.csv', index=False)


In [3]:
!pip install dash

In [20]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# Decompose using statsmodels (as before)
from statsmodels.tsa.seasonal import seasonal_decompose

# Make sure to replace 'path_to_your_data.csv' with the actual path to your dataset.
df = pd.read_csv('/content/avocado_processed (1).csv')

# Data preprocessing
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

df = df.set_index('date')

# Resample the data to monthly frequency and sum the 'total_volume'
monthly_sales = df['total_volume'].resample('M').sum()
decomposition = seasonal_decompose(monthly_sales, model='additive', period=12)
# Create a Dash application instance
app = dash.Dash(__name__)

# Layout of the Dash application
app.layout = html.Div(style={'backgroundColor': '#1E1E1E', 'color': 'white', 'padding': '50px'}, children=[
    html.H1(children='Sales Forecasting Dashboard'),

    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': str(year), 'value': year} for year in df['year'].unique()],
        value=df['year'].max(),
        clearable=False,
        style={'width': '50%', 'color': 'black'}
    ),
    dcc.Graph(id='sales-pie-chart'),
    dcc.Graph(id='sales-trends-chart'),
    dcc.Graph(id='sales-region-bar-chart'),
    dcc.Graph(id='sales-relationship-chart'),
    dcc.Graph(id='sales-relationship-chart-1'),

])

# Callbacks to update the dashboard based on the selected year
@app.callback(
    Output('sales-pie-chart', 'figure'),
    Output('sales-trends-chart', 'figure'),
    Output('sales-region-bar-chart', 'figure'),
    Output('sales-relationship-chart', 'figure'),
    Output('sales-relationship-chart-1', 'figure'),
    Input('year-dropdown', 'value')
)
def update_dashboard(selected_year):
    sales_by_region = df.groupby('geography')['total_volume'].sum().reset_index()
    pie_fig = px.pie(sales_by_region, values='total_volume', names='geography',
                 title='Avocado Sales by Region',
                )

    line_monthly_fig = go.Figure()

    line_monthly_fig.add_trace(go.Scatter(x=monthly_sales.index, y=monthly_sales.values,
                            mode='lines', name='Monthly Sales'))

    line_monthly_fig.update_layout(title='Monthly Avocado Sales',
                      xaxis_title='Date',
                      yaxis_title='Total Volume')
    # Create subplots for decomposition components
    line_fig = make_subplots(rows=4, cols=1, subplot_titles=('Observed', 'Trend', 'Seasonal', 'Residuals'))

    # Add traces for observed data
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.observed, mode='lines', name='Observed'), row=1, col=1)
    # Add traces for trend
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.trend, mode='lines', name='Trend'), row=2, col=1)
    # Add traces for seasonal component
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.seasonal, mode='lines', name='Seasonal'), row=3, col=1)
    # Add traces for residuals
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.resid, mode='lines', name='Residuals'), row=4, col=1)

    line_fig.update_layout(height=800, title_text="Time Series Decomposition")

    #Average Price by Season
    season_price = df.groupby('season')['average_price'].mean().reset_index()
    bar_fig = px.bar(season_price, x='season', y='average_price', title='Average Price by Season',
                  labels={'season': 'Season', 'average_price': 'Average Price'})
    #Total Sales by Season
    season_sales = df.groupby('season')['total_volume'].sum().reset_index()
    fig4 = make_subplots(rows=1, cols=2, subplot_titles=('Total Sales by Season', 'Average Price by Season'))
    fig4.add_trace(go.Bar(x=season_sales['season'], y=season_sales['total_volume'], name='Total Sales'), row=1, col=1)
    fig4.add_trace(go.Bar(x=season_price['season'], y=season_price['average_price'], name='Average Price'), row=1, col=2)
    fig4.update_layout(title='Total Sales and Average Price by Season')
    #Volume Price Interaction
    fig5 = make_subplots(rows=1, cols=2, subplot_titles=('Average Volume Price by Promotion','Sales with and without Promotions'))
    volume_price_interaction = df.groupby('promotion')['volume_price_interaction'].mean().reset_index()
    fig5.add_trace(go.Bar(x=volume_price_interaction['promotion'], y=volume_price_interaction['volume_price_interaction'], name='Average Volume Price Interaction'), row=1, col=1)

    promotion_interaction = df.groupby('promotion')['volume_price_interaction'].mean().reset_index()
    fig5.add_trace(go.Bar(x=promotion_interaction['promotion'], y=promotion_interaction['volume_price_interaction'], name='Sales with and without Promotions'), row=1, col=2)
    fig5.update_layout(title='Average Volume Price Interaction by Promotion')

    return pie_fig, line_fig ,line_monthly_fig, fig4, fig5

# Run app
if __name__ == '__main__':
    app.run(debug=True)

<ipython-input-20-777249092e72>:21: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



<IPython.core.display.Javascript object>

In [ ]:
selected_year = 2020
filtered_df = df[df['year'] == selected_year]
previous_year = selected_year - 1
previous_year_df = df[df['year'] == previous_year]

# Calculate YOY Growth
current_sales = filtered_df['total_volume'].sum()
previous_sales = previous_year_df['total_volume'].sum()
yoy_growth = ((current_sales - previous_sales) / previous_sales * 100) if previous_sales != 0 else 0

year_info = f'Current Year: {selected_year} | Previous Year: {previous_year} | YOY Growth: {yoy_growth:.2f}%'
"""
# Create pie chart showing sales by type
sales_type_df = filtered_df.groupby('type')['total_volume'].sum().reset_index()
pie_fig = px.pie(sales_type_df, names='type', values='total_volume', title='Sales by Type')
"""
sales_by_region = df.groupby('geography')['total_volume'].sum().reset_index()
pie_fig = px.pie(sales_by_region, values='total_volume', names='geography',
             title='Avocado Sales by Region',
            )
"""# Create timeline showing sales trends
sales_trends_df = filtered_df.groupby(['month'])['total_volume'].sum().reset_index()
line_fig = px.line(sales_trends_df, x='month', y='total_volume', title='Sales Trends Over Months')
"""
line_monthly_fig = go.Figure()

line_monthly_fig.add_trace(go.Scatter(x=monthly_sales.index, y=monthly_sales.values,
                         mode='lines', name='Monthly Sales'))

line_monthly_fig.update_layout(title='Monthly Avocado Sales',
                  xaxis_title='Date',
                  yaxis_title='Total Volume')
# Create subplots for decomposition components
line_fig = make_subplots(rows=4, cols=1, subplot_titles=('Observed', 'Trend', 'Seasonal', 'Residuals'))

# Add traces for observed data
line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.observed, mode='lines', name='Observed'), row=1, col=1)
# Add traces for trend
line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.trend, mode='lines', name='Trend'), row=2, col=1)
# Add traces for seasonal component
line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.seasonal, mode='lines', name='Seasonal'), row=3, col=1)
# Add traces for residuals
line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.resid, mode='lines', name='Residuals'), row=4, col=1)

line_fig.update_layout(height=800, title_text="Time Series Decomposition")

"""# Bar chart for sales by region
sales_region_df = filtered_df.groupby('region')['total_volume'].sum().reset_index()
bar_fig = px.bar(sales_region_df, x='region', y='total_volume', title='Sales by Region')"""

#Average Price by Season
season_price = df.groupby('season')['average_price'].mean().reset_index()
bar_fig = px.bar(season_price, x='season', y='average_price', title='Average Price by Season',
              labels={'season': 'Season', 'average_price': 'Average Price'})

# Scatter plot showing relationships (example: volume vs average price)
scatter_fig = px.scatter(filtered_df, x='average_price', y='total_volume', color='type',
                          title='Relationship between Sales Volume and Average Price')

print(year_info)
pie_fig.show()
line_monthly_fig.show()
line_fig.show()
bar_fig.show()
scatter_fig.show()

,average_price,total_volume,4046,4225,4770,total_bags,type,year,geography,is_holiday,...,month,season,weekday_name,is_christmas_season,promotion_effect,region,volume_price_interaction,promotion_interaction,month_change,monthly_avg_volume
date,,,,,,,,,,,,,,,,,,,,,
2015-01-04,1.22,40873.28,2819.50,28287.42,49.900000,9716.46,conventional,2015,Albany,0,...,1,Winter,Sunday,0,0.0,Region B,49865.4016,0.0,NaN,199834.613920
2015-01-04,1.79,1373.95,57.42,153.88,21629.374703,1162.65,organic,2015,Albany,0,...,1,Winter,Sunday,0,0.0,Region B,2459.3705,0.0,-0.966385,199834.613920
2015-01-04,1.00,435021.49,364302.39,23821.16,82.150000,46815.79,conventional,2015,Atlanta,0,...,1,Winter,Sunday,0,0.0,Region B,435021.4900,0.0,315.621049,199834.613920
2015-01-04,1.76,3846.69,1500.15,938.35,21629.374703,1408.19,organic,2015,Atlanta,0,...,1,Winter,Sunday,0,0.0,Region B,6770.1744,0.0,-0.991157,199834.613920
2015-01-04,1.08,788025.06,53987.31,552906.04,39995.030000,141136.68,conventional,2015,Baltimore/Washington,0,...,1,Winter,Sunday,0,0.0,Region B,851067.0648,0.0,203.857958,199834.613920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-17,1.58,124204.74,150100.00,198457.00,5429.000000,1917250.00,organic,2020,Total U.S.,0,...,5,Spring,Sunday,0,0.0,Region B,196243.4892,0.0,0.000000,205458.239896
2020-05-17,1.09,124204.74,2081824.04,1020965.12,33410.850000,5531562.87,conventional,2020,West,0,...,5,Spring,Sunday,0,0.0,Region B,135383.1666,0.0,0.000000,205458.239896
2020-05-17,1.71,384158.00,23455.00,39738.00,1034.000000,319932.00,organic,2020,West,0,...,5,Spring,Sunday,0,0.0,Region B,656910.1800,0.0,2.092942,205458.239896


In [76]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# Decompose using statsmodels (as before)
from statsmodels.tsa.seasonal import seasonal_decompose

# Make sure to replace 'path_to_your_data.csv' with the actual path to your dataset.
df = pd.read_csv('/content/avocado_processed (1).csv')

# Data preprocessing
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

df = df.set_index('date')

# Resample the data to monthly frequency and sum the 'total_volume'
monthly_sales = df['total_volume'].resample('M').sum()
decomposition = seasonal_decompose(monthly_sales, model='additive', period=12)
# Create a Dash application instance
app = dash.Dash(__name__)

# Layout of the Dash application
app.layout = html.Div(style={'backgroundColor': '#1E1E1E', 'color': 'white', 'padding': '50px'}, children=[
    html.H1(children='Sales Forecasting Dashboard'),
    html.Hr(style={'border': '1px solid white'}),  # Add a separator line between dropdowns and graphs
    html.Div(id='year-info'),
    dcc.Graph(id='sales-pie-chart'),
    dcc.Graph(id='sales-trends-chart'),
    dcc.Graph(id='sales-region-bar-chart'),
    dcc.Graph(id='sales-relationship-chart'),
    dcc.Graph(id='sales-relationship-chart-1'),
    dcc.Graph(id='sales-relationship-chart-2'),
    html.H1(children='Sales For Selected Country'),
    # Fix the dropdowns syntax
    html.Div([
        dcc.Dropdown(
            id='year-dropdown',
            options=[{'label': str(year), 'value': year} for year in df['year'].unique()],
            value=df['year'].max(),
            clearable=False,
            style={'width': '48%', 'color': 'black'}
        ),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': str(geo), 'value': geo} for geo in df['geography'].unique()],
            value=df['geography'].iloc[0],
            clearable=False,
            style={'width': '48%', 'color': 'black'}
        )
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),  # <== Add the comma here!

    html.Hr(style={'border': '1px solid white'}),  # Add a separator line between dropdowns and graphs
    dcc.Graph(id='sales-relationship-chart-3'),
    dcc.Graph(id='sales-relationship-chart-4'),
    dcc.Graph(id='sales-relationship-chart-5'),
    dcc.Graph(id='sales-relationship-chart-6'),
])


# Callbacks to update the dashboard based on the selected year
@app.callback(
    Output('sales-pie-chart', 'figure'),
    Output('sales-trends-chart', 'figure'),
    Output('sales-region-bar-chart', 'figure'),
    Output('sales-relationship-chart', 'figure'),
    Output('sales-relationship-chart-1', 'figure'),
    Output('sales-relationship-chart-2', 'figure'),
    Output('sales-relationship-chart-3', 'figure'),
    Output('sales-relationship-chart-4', 'figure'),
    Output('sales-relationship-chart-5', 'figure'),
    Output('sales-relationship-chart-6', 'figure'),
    Input('year-dropdown', 'value')
    ,Input('country-dropdown', 'value')
)
def update_dashboard(selected_year,selected_country):
    sales_by_region = df.groupby('geography')['total_volume'].sum().reset_index()
    pie_fig = px.pie(sales_by_region, values='total_volume', names='geography',
                 title='Avocado Sales by Region',
                )

    line_monthly_fig = go.Figure()

    line_monthly_fig.add_trace(go.Scatter(x=monthly_sales.index, y=monthly_sales.values,
                            mode='lines', name='Monthly Sales'))

    line_monthly_fig.update_layout(title='Monthly Avocado Sales',
                      xaxis_title='Date',
                      yaxis_title='Total Volume')
    # Create subplots for decomposition components
    line_fig = make_subplots(rows=4, cols=1, subplot_titles=('Observed', 'Trend', 'Seasonal', 'Residuals'))

    # Add traces for observed data
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.observed, mode='lines', name='Observed'), row=1, col=1)
    # Add traces for trend
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.trend, mode='lines', name='Trend'), row=2, col=1)
    # Add traces for seasonal component
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.seasonal, mode='lines', name='Seasonal'), row=3, col=1)
    # Add traces for residuals
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.resid, mode='lines', name='Residuals'), row=4, col=1)

    line_fig.update_layout(height=800, title_text="Time Series Decomposition")

    #Average Price by Season
    season_price = df.groupby('season')['average_price'].mean().reset_index()
    bar_fig = px.bar(season_price, x='season', y='average_price', title='Average Price by Season',
                  labels={'season': 'Season', 'average_price': 'Average Price'})
    #Total Sales by Season
    season_sales = df.groupby('season')['total_volume'].sum().reset_index()
    fig4 = make_subplots(rows=1, cols=2, subplot_titles=('Total Sales by Season', 'Average Price by Season'))
    fig4.add_trace(go.Bar(x=season_sales['season'], y=season_sales['total_volume'], name='Total Sales'), row=1, col=1)
    fig4.add_trace(go.Bar(x=season_price['season'], y=season_price['average_price'], name='Average Price'), row=1, col=2)
    fig4.update_layout(title='Total Sales and Average Price by Season')
    #Volume Price Interaction
    fig5 = make_subplots(rows=1, cols=2, subplot_titles=('Average Volume Price by Promotion','Sales with and without Promotions'))
    volume_price_interaction = df.groupby('promotion')['volume_price_interaction'].mean().reset_index()
    fig5.add_trace(go.Bar(x=volume_price_interaction['promotion'], y=volume_price_interaction['volume_price_interaction'], name='Average Volume Price Interaction'), row=1, col=1)

    promotion_interaction = df.groupby('promotion')['volume_price_interaction'].mean().reset_index()
    fig5.add_trace(go.Bar(x=promotion_interaction['promotion'], y=promotion_interaction['volume_price_interaction'], name='Sales with and without Promotions'), row=1, col=2)
    fig5.update_layout(title='Average Volume Price Interaction by Promotion')

    fig6 = go.Figure()
    fig6.add_trace(go.Line(
        x=df.index,
        y=df['total_volume'],
        name="Sales"
    ))
    fig6.add_trace(go.Line(
        x=df.index,
        y=df['promotion'] * df['total_volume'].max(),
        name="promotion"
    ))
    fig6.update_layout(
        title="Sales and Promotions over Time",title_x=0.5,
        xaxis_title="Date",
        yaxis_title="Sales Volume"
    )
    #
    dff_Ctry=df[df["geography"]==selected_country]      # Filter Countries
    dff_temp_year=dff_Ctry[dff_Ctry["year"]==selected_year]        # filter Year Also selected by user
    dff_temp_year=dff_temp_year.sort_values(by='month')
    fig7 = px.bar(dff_temp_year,x='month', y="monthly_avg_volume")

    fig8 = px.line(dff_temp_year,x='month', y="month_change")
    fig9 = px.bar(dff_Ctry,x='year', y="total_volume")
    fig10 =px.pie(dff_Ctry,values='total_volume',names='weekday_name')
    return pie_fig, line_fig ,line_monthly_fig, fig4, fig5, fig6, fig7, fig8, fig9, fig10

# Run app
if __name__ == '__main__':
    app.run(debug=True)

<ipython-input-76-47e7b7d6a098>:21: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



<IPython.core.display.Javascript object>

In [44]:

selected_country = df['geography'][0]

dff_Ctry=df[df["geography"]==selected_country]      # Filter Countries
dff_temp_year=dff_Ctry[dff_Ctry["year"]==2015]        # filter Year Also selected by user
dff_temp_year=dff_temp_year.sort_values(by='month')
fig = px.bar(dff_temp_year,x='month', y="monthly_avg_volume")
fig.show()

<ipython-input-44-0956c133cf89>:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

